<a href="https://colab.research.google.com/github/MTia05/Visualizzazione-Scientifica/blob/main/codici/confrontoFumatoriProfessione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confronto Fumatori per Posizione professionale

## Grafico a barre verticali

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Percorso
# Trova la cartella dove si trova il notebook
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Unisce i pezzi in modo corretto
file_path = os.path.join(base_path, "dataSet", "datasetFumatoriProfessione(istogramma).csv")

# Colori
COLOR_2024 = "#5DADE2"    # Blu cielo
COLOR_TEXT = "#2E4053"    # Colore per il titolo principale
COLOR_SUB  = "#95A5A6"    # Colore per sottotitoli e assi

try:
    # # Il file ha il punto e virgola come separatore!
    df = pd.read_csv(file_path, sep=';', encoding='latin-1')

    # Pulizia nomi colonne e dati
    df.columns = [c.strip().replace("'", "") for c in df.columns]

    # Identificazione colonne
    status_col = 'LABOUR_PROFESS_STATUS_B'
    cat_col = 'Condizione e posizione nella professione'
    val_col = 'Osservazione'
    year_col = 'TIME_PERIOD'

    # Pulizia dei testi
    df[cat_col] = df[cat_col].astype(str).str.replace("'", "").str.strip()
    df[status_col] = df[status_col].astype(str).str.replace("'", "").str.strip()

    # Conversione numerica del valore
    df[val_col] = pd.to_numeric(df[val_col], errors='coerce')

    # Lista delle 11 categorie target
    target_codes = ['1', '1A', '1B', '1C', '1D', '2', '3', '4', '5', '6', '7']

    # Solo anno 2024 e solo i codici richiesti
    final_data = df[
        (df[year_col] == 2024) &
        (df[status_col].isin(target_codes))
    ].sort_values(by=status_col)

    # Estetica
    sns.set_theme(style="white")
    fig, ax = plt.subplots(figsize=(14, 8), dpi=100)

    # Dati per il plot
    categories = final_data[cat_col].values
    vals = final_data[val_col].values
    x = np.arange(len(categories))

    # Barre sottili
    rects = ax.bar(x, vals, width=0.5, color=COLOR_2024, zorder=3)

    # Punto alto e suffisso k
    def format_high_dot_k(value):
        if int(value) == 0:
            return '0' # 0 senza k se il valore è zero
        formatted = f'{int(value):,}'
        return formatted.replace(',', '˙') + 'k'

    # Annotazioni
    for rect in rects:
        height = rect.get_height()
        ax.annotate(format_high_dot_k(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontsize=10, fontweight='bold', color=COLOR_TEXT)

    # Titolo
    plt.title('Fumatori in Italia per Condizione Professionale',
              fontsize=22, pad=40, fontweight='bold', color=COLOR_TEXT, loc='left')

    # Sottotitolo
    ax.text(0, 1.05, f'Dati del 2024 • Valori espressi in migliaia',
            transform=ax.transAxes, fontsize=12, color=COLOR_SUB)

    # Nomi dell'asse
    plt.ylabel('Fumatori (migliaia)', fontsize=11, labelpad=15, color=COLOR_SUB)

    # Layout
    sns.despine(left=True, bottom=True)
    ax.yaxis.grid(True, linestyle='-', alpha=0.1, color='black')

    ax.set_xticks(x)
    # Rotazione dei nomi
    ax.set_xticklabels(categories, rotation=30, ha='right', fontsize=9, color=COLOR_SUB)

    # Formattazione dell'asse y
    def y_axis_formatter(x, pos):
        return format_high_dot_k(x)
    ax.yaxis.set_major_formatter(plt.FuncFormatter(y_axis_formatter))

    plt.yticks(fontsize=10, color=COLOR_SUB)

    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"Errore durante l'elaborazione: {e}")

## Grafico a inee

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import os

# Percorso
# Trova la cartella dove si trova il notebook
base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Unisce i pezzi in modo corretto
file_path = os.path.join(base_path, "dataSet", "datasetFumatoriProfessione(linee).csv")

try:
    # Caricament
    df = pd.read_csv(
        file_path,
        sep=';',
        encoding='latin-1',
        quoting=csv.QUOTE_NONE,
        on_bad_lines='skip'
    )

    # Pulizia nomi colonne
    df.columns = [c.strip().replace('"', '').replace("'", "") for c in df.columns]

    # Mappatura indici



    col_id_idx = 8      # Indice 8 ID
    col_anno_idx = 10   # Indice 10 Anno
    col_valore_idx = 11 # Indice 11: Valore

    # Estrazione dati
    raw_data = pd.DataFrame({
        'ID': df.iloc[:, col_id_idx].astype(str).str.strip(),
        'Anno': pd.to_numeric(df.iloc[:, col_anno_idx], errors='coerce'),
        'Valore': pd.to_numeric(df.iloc[:, col_valore_idx], errors='coerce')
    }).dropna()

    # Raggruppamento dati
    # 1: Occupati
    # 2: Disoccupati 1.0
    # 3: Disoccupati 2.0
    # 4, 5, 6, 7: Non-occupati (Casalinghe, studenti, ritirati, altri)
    def map_categories(row_id):
        if row_id == '1':
            return 'Occupati'
        if row_id == '2':
            return 'Disoccupati (nuova occ.)'
        if row_id == '3':
            return 'Disoccupati (prima occ.)'
        if row_id in ['4', '5', '6', '7']:
            return 'Inattivi / Altri'
        return None

    raw_data['Categoria'] = raw_data['ID'].apply(map_categories)

    # Solo le righe valide
    filtered_data = raw_data[raw_data['Categoria'].notnull()].copy()

    # Raggruppamento per Anno e Categoria
    final_data = filtered_data.groupby(['Anno', 'Categoria'])['Valore'].sum().reset_index()
    final_data = final_data.sort_values(by=['Anno'])

    # Conversione in milioni
    final_data['Valore_M'] = final_data['Valore'] / 1000

    # Estetica
    sns.set_theme(style="white")
    fig, ax = plt.subplots(figsize=(14, 9), dpi=100)

    # Palette colori
    colors = {
        'Occupati': '#5DADE2',              # Blu
        'Disoccupati (nuova occ.)': '#E74C3C', # Rosso
        'Disoccupati (prima occ.)': '#8E44AD', # Viola
        'Inattivi / Altri': '#F39C12'        # Arancione
    }

    # Ordine categorie
    categories = ['Occupati', 'Disoccupati (nuova occ.)', 'Disoccupati (prima occ.)', 'Inattivi / Altri']

    for cat in categories:
        cat_df = final_data[final_data['Categoria'] == cat]
        if not cat_df.empty:
            plt.plot(
                cat_df['Anno'],
                cat_df['Valore_M'],
                label=cat,
                color=colors[cat],
                linewidth=2.5,
                marker='o',
                markersize=6,
                zorder=3
            )

            # Valori in milioni M
            for x, y in zip(cat_df['Anno'], cat_df['Valore_M']):
                ax.annotate(
                    f'{y:.1f}M',
                    (x, y),
                    textcoords="offset points",
                    xytext=(0, 10),
                    ha='center',
                    fontsize=8,
                    fontweight='bold',
                    color=colors[cat]
                )

    # Titolo
    plt.title('Andamento Fumatori in Italia per Condizione Professionale',
              fontsize=22, pad=55, fontweight='bold', color='#2E4053', loc='left')

    # Sottotitolo
    ax.text(0, 1.07, 'Dati dal 2001 al 2024 • Valori espressi in milioni',
            transform=ax.transAxes, fontsize=12, color='#95A5A6', fontweight='normal')

    # Assi
    plt.xlabel('Anno di rilevazione', fontsize=11, labelpad=15, color='#95A5A6')
    plt.ylabel('Fumatori (milioni)', fontsize=11, labelpad=15, color='#95A5A6')

    # Legenda
    plt.legend(frameon=False, fontsize=10, loc='upper left', bbox_to_anchor=(0, 1.05), ncol=4)

    # Bordi e griglia
    sns.despine(left=True, bottom=True)
    ax.yaxis.grid(True, linestyle='-', alpha=0.1, color='black')

    # Tick
    years = sorted(final_data['Anno'].unique().astype(int))
    plt.xticks(years, rotation=45, fontsize=10, color='#95A5A6')
    plt.yticks(fontsize=10, color='#95A5A6')

    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"Errore durante l'elaborazione: {e}")